<a href="https://colab.research.google.com/github/noelparakkal/NectarInternship/blob/master/jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import io
from google.colab import files
from IPython.display import Javascript
import numpy as np
from collections import defaultdict

def convert_to_serializable(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    if isinstance(obj, (np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, pd.Timestamp):
        return obj.strftime('%Y-%m-%d %H:%M:%S')
    return obj

def analyze_assets_comprehensive():
    # Step 1: Upload Excel file
    print("📤 Please upload your Excel file:")
    uploaded = files.upload()
    file_name = next(iter(uploaded))

    try:
        # Step 2: Process data starting from row 4
        xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))
        df = pd.read_excel(xls, header=None, skiprows=3)

        # Set column names from row 3
        df.columns = df.iloc[0].str.strip().str.upper()
        df = df[1:].reset_index(drop=True)

        # Validate required columns
        required_columns = [
            'ASSET NAME', 'PRIORITY', 'JOB TYPE', 'SLA MET', 'REQUEST TIME',
            'COMMUNITY', 'SUB COMMUNITY', 'BUILDING', 'JOB LOCATION NAME',
            'ASSIGNEE', 'VENDOR', 'MANAGED BY'
        ]
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {', '.join(missing_cols)}")

        # Convert REQUEST TIME to datetime
        df['REQUEST TIME'] = pd.to_datetime(df['REQUEST TIME'], format='%d %b %Y %H:%M:%S', errors='coerce')


        # Extract year and month for monthly analysis
        df['YEAR'] = df['REQUEST TIME'].dt.year
        df['MONTH'] = df['REQUEST TIME'].dt.month
        df['YEAR_MONTH'] = df['REQUEST TIME'].dt.to_period('M')

        # Clean SLA data
        df['SLA MET'] = df['SLA MET'].astype(str).str.strip().str.upper()

        # Function to count SLA status
        def count_sla_status(series):
            passed = series.str.contains('PASS|MET|YES|TRUE|1|SLA-PASSED', case=False, regex=True).sum()
            failed = series.str.contains('FAIL|NO|FALSE|0|SLA-FAILED', case=False, regex=True).sum()
            return passed, failed

        # Step 3: Get top 20 assets by job count
        top_assets = df['ASSET NAME'].value_counts().head(20).index.tolist()

        # Step 4: Comprehensive analysis
        results = {
            "analysis_date": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            "source_file": file_name,
            "top_assets": [],
            "monthly_request_patterns": {}  # New section for monthly request patterns
        }

        # New: Analyze monthly request patterns for all assets (not just top 20)
        monthly_patterns = defaultdict(lambda: defaultdict(list))

        for _, row in df.iterrows():
            asset = row['ASSET NAME']
            year_month = row['YEAR_MONTH'].strftime('%Y-%m')
            monthly_patterns[asset][year_month].append(row['REQUEST TIME'].strftime('%Y-%m-%d %H:%M:%S'))

        # Convert to regular dict and sort
        monthly_patterns = {
            asset: dict(sorted(months.items()))
            for asset, months in monthly_patterns.items()
        }

        # Add to results
        results['monthly_request_patterns'] = monthly_patterns

        for rank, asset in enumerate(top_assets, 1):
            asset_data = df[df['ASSET NAME'] == asset]

            # Location details
            location = asset_data.iloc[0][['COMMUNITY', 'SUB COMMUNITY', 'BUILDING', 'JOB LOCATION NAME']].fillna('').to_dict()

            # Technician/Vendor details
            assignee_dist = asset_data['ASSIGNEE'].value_counts().to_dict()
            vendor_dist = asset_data['VENDOR'].value_counts().to_dict()
            managed_by_dist = asset_data['MANAGED BY'].value_counts().to_dict()

            # SLA analysis
            sla_passed, sla_failed = count_sla_status(asset_data['SLA MET'])
            unclear_sla = len(asset_data) - sla_passed - sla_failed
            sla_pass_percentage = round(sla_passed/len(asset_data)*100, 2) if len(asset_data) > 0 else 0

            # Priority and job type breakdown
            priority_dist = {}
            for priority in asset_data['PRIORITY'].unique():
                priority_data = asset_data[asset_data['PRIORITY'] == priority]
                job_type_counts = priority_data['JOB TYPE'].value_counts().to_dict()

                priority_dist[priority] = {
                    "count": len(priority_data),
                    "percentage": round(len(priority_data)/len(asset_data)*100, 1),
                    "job_types": job_type_counts,
                    "job_type_percentages": {
                        k: round(v/len(priority_data)*100, 1)
                        for k, v in job_type_counts.items()
                    }
                }

            # Monthly request frequency for this asset
            asset_monthly = asset_data.groupby('YEAR_MONTH').size().reset_index(name='count')
            asset_monthly['YEAR_MONTH'] = asset_monthly['YEAR_MONTH'].astype(str)
            monthly_freq = asset_monthly.to_dict('records')

            # Convert all numpy types to native Python types
            asset_info = {
                "rank": int(rank),
                "asset_name": str(asset),
                "total_job_requests": int(len(asset_data)),
                "monthly_request_frequency": monthly_freq,  # New field
                "location": {k: str(v) for k, v in location.items()},
                "assignee_distribution": {str(k): int(v) for k, v in assignee_dist.items()},
                "vendor_distribution": {str(k): int(v) for k, v in vendor_dist.items()},
                "managed_by_distribution": {str(k): int(v) for k, v in managed_by_dist.items()},
                "sla_metrics": {
                    "passed": int(sla_passed),
                    "failed": int(sla_failed),
                    "unclear": int(unclear_sla),
                    "pass_percentage": float(sla_pass_percentage)
                },
                "priority_distribution": {
                    str(k): {
                        "count": int(v["count"]),
                        "percentage": float(v["percentage"]),
                        "job_types": {str(k2): int(v2) for k2, v2 in v["job_types"].items()},
                        "job_type_percentages": {str(k2): float(v2) for k2, v2 in v["job_type_percentages"].items()}
                    }
                    for k, v in priority_dist.items()
                }
            }

            # Append comprehensive asset analysis
            results['top_assets'].append(asset_info)

        # Convert numpy types in the root level
        results = json.loads(json.dumps(results, default=convert_to_serializable))

        # Step 5: Generate JSON and download
        json_output = json.dumps(results, indent=4)
        json_filename = f"comprehensive_asset_analysis_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"

        # Auto-download JSON
        print("⏳ Preparing download...")
        display(Javascript(f"""
            const content = {json.dumps(results)};
            const blob = new Blob([JSON.stringify(content, null, 2)], {{type: 'application/json'}});
            const a = document.createElement('a');
            a.href = URL.createObjectURL(blob);
            a.download = '{json_filename}';
            document.body.appendChild(a);
            a.click();
            document.body.removeChild(a);
            """))

        print(f"✅ Analysis complete! Downloaded '{json_filename}'")

        # Preview in console with the exact format requested
        print("\n🔍 Top Assets Comprehensive Analysis:")
        for asset in results['top_assets'][:5]:  # Show top 5 for preview
            print(f"\n{asset['rank']}. {asset['asset_name']} ({asset['total_job_requests']} requests)")

            # Location
            loc = asset['location']
            location_str = f"{loc['COMMUNITY']} > {loc['SUB COMMUNITY']} > {loc['BUILDING']}"
            if loc['JOB LOCATION NAME']:
                location_str += f" > {loc['JOB LOCATION NAME']}"
            print(f"📍 Location: {location_str}")

            # Monthly frequency
            print("\n📅 Monthly Request Frequency:")
            for month in asset['monthly_request_frequency']:
                print(f"   - {month['YEAR_MONTH']}: {month['count']} requests")

            # Show sample request timestamps (first 3 months)
            asset_patterns = results['monthly_request_patterns'].get(asset['asset_name'], {})
            for i, (month, timestamps) in enumerate(asset_patterns.items()):
                if i >= 3:  # Limit to first 3 months for preview
                    break
                print(f"\n   🕒 Request times for {month} (first 5 of {len(timestamps)}):")
                for ts in timestamps[:5]:
                    print(f"      - {ts}")
                if len(timestamps) > 5:
                    print(f"      ... and {len(timestamps)-5} more")

            # Assignees
            if asset['assignee_distribution']:
                print("\n👷 Assignees:")
                for assignee, count in asset['assignee_distribution'].items():
                    print(f"   - {assignee}: {count}")

            # SLA Metrics
            print(f"\n{asset['asset_name']}")
            print(f"   Total Requests: {asset['total_job_requests']}")
            print(f"   SLA Passed: {asset['sla_metrics']['passed']} ({asset['sla_metrics']['pass_percentage']}%)")

        return json_output

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Run the comprehensive analysis
analysis_results = analyze_assets_comprehensive()

📤 Please upload your Excel file:


Saving Jobs 1001-2000 (3).xlsx to Jobs 1001-2000 (3).xlsx
⏳ Preparing download...


<IPython.core.display.Javascript object>

✅ Analysis complete! Downloaded 'comprehensive_asset_analysis_20250515_114301.json'

🔍 Top Assets Comprehensive Analysis:

1. BT A SPF 11 (2 requests)
📍 Location: Jumeirah Village Circle > Bloom Towers > Bloom Towers A

📅 Monthly Request Frequency:
   - 2025-01: 1 requests
   - 2025-02: 1 requests

   🕒 Request times for 2025-01 (first 5 of 1):
      - 2025-01-15 04:00:00

   🕒 Request times for 2025-02 (first 5 of 1):
      - 2025-02-15 04:00:00

👷 Assignees:
   - HVAC TechnicianThree: 1
   - HVAC TechnicianFour: 1

BT A SPF 11
   Total Requests: 2
   SLA Passed: 1 (50.0%)

2. BT C 32F FanCoilUnit 70 (2 requests)
📍 Location: Jumeirah Village Circle > Bloom Towers > Bloom Towers C

📅 Monthly Request Frequency:
   - 2025-02: 1 requests
   - 2025-03: 1 requests

   🕒 Request times for 2025-02 (first 5 of 1):
      - 2025-02-18 06:25:25

   🕒 Request times for 2025-03 (first 5 of 1):
      - 2025-03-01 04:00:00

👷 Assignees:
   - HVAC TechnicianThree: 2

BT C 32F FanCoilUnit 70
   Total R